In [2]:
import sys
import json
import os
import requests
import random
from datetime import datetime
import time

import pandas as pd
import numpy as np
import sklearn.linear_model

# host = ""

def load_digits(digits_location, digits_filename = "train.data", norm=True):
    digits_path = digits_location + "/" + digits_filename
    print("Source file: %s" % digits_path)
    df = pd.read_csv(digits_path, sep=",", header=None)
    data = df.values
    print("Number of image files: %d" % len(data))
    y = data[:,0]
    X = data[:,1:]
    Z = X
    if norm:
        mu = np.mean(X,0)
        sigma = np.var(X,0)
        Z = (X - mu) / np.array([np.sqrt(z) if z > 0 else 1. for z in sigma])
    return Z, y

def filter_data(data):
    cx, cy = data
    binary_x = []
    binary_y = []
    for i in range(len(cy)):
        if cy[i] == 3:
            binary_x.append(cx[i,:])
            binary_y.append(1.0)
        elif cy[i] == 6:
            binary_x.append(cx[i,:])
            binary_y.append(0.0)
    return np.array(binary_x), np.array(binary_y)
            

def mnist_update(host, uid, x, y):
    url = "http://%s:1337/update" % host
    req_json = json.dumps({'uid': uid, 'input': list(x), 'label': y})
    headers = {'Content-type': 'application/json'}
    start = datetime.now()
    r = requests.post(url, headers=headers, data=req_json)
    end = datetime.now()
    latency = (end - start).total_seconds() * 1000.0
    print("'%s', %f ms" % (r.text, latency))

def mnist_prediction(host, uid, x):
    url = "http://%s:1337/predict" % host
    req_json = json.dumps({'uid': uid, 'input': list(x)})
    headers = {'Content-type': 'application/json'}
    # x_str = ", ".join(["%d" % a for a in x])
    start = datetime.now()
    r = requests.post(url, headers=headers, data=req_json)
    end = datetime.now()
    latency = (end - start).total_seconds() * 1000.0
    pred = float(r.text.split(":")[1].strip())
    return (pred, latency)



def send_updates(num_updates):
    uid = 0
    for i in range(num_updates):
        example_num = np.random.randint(0, len(test_y))
        mnist_update(host, uid, test_x[example_num], float(test_y[example_num]))
    

def run_serving_workload():
    uid = 0
    while True:
        correct = 0
        false_pos = 0
        false_neg = 0
        latencies = []
        true_pos = 0
        true_neg = 0
        for i in range(100):
            example_num = np.random.randint(0, len(test_y))
            correct_y = float(test_y[example_num])
            pred_y, latency = mnist_prediction(host, uid, test_x[example_num])
            if correct_y == pred_y:
                if correct_y == 0:
                    true_neg += 1
                elif correct_y == 1:
                    true_pos += 1
                correct += 1
            elif correct_y == 0 and pred_y == 1:
                false_pos += 1
            elif correct_y == 1 and pred_y == 0:
                false_neg += 1
            else:
                print "predicted: {p}, correct: {c}".format(p=pred_y, c=correct_y)
            latencies.append(latency)
        print "Accuracy: {cor}, false positives: {fp}, false negatives: {fn}, true positives: {tp}, true negatives: {tn}".format(
            cor = float(correct)/100.0,
            fp=float(false_pos)/100.0,
            fn=float(false_neg)/100.0,
            tp=float(true_pos)/100.0,
            tn=float(true_neg)/100.0)
        print "Mean latency: {lat} ms".format(lat = np.mean(latencies))
            
    


test_x, test_y = filter_data(load_digits(os.path.expanduser("~/model-serving/data/mnist_data"), digits_filename="test.data"))


Source file: /Users/crankshaw/model-serving/data/mnist_data/test.data
Number of image files: 10000


In [3]:
host = "ec2-52-53-151-0.us-west-1.compute.amazonaws.com"
send_updates(50)
#run_serving_workload()

'Update scheduled', 47.004000 ms
'Update scheduled', 31.275000 ms
'Update scheduled', 37.661000 ms
'Update scheduled', 31.133000 ms
'Update scheduled', 62.385000 ms
'Update scheduled', 48.323000 ms
'Update scheduled', 31.603000 ms
'Update scheduled', 31.971000 ms
'Update scheduled', 32.284000 ms
'Update scheduled', 38.884000 ms
'Update scheduled', 36.117000 ms
'Update scheduled', 44.089000 ms
'Update scheduled', 34.916000 ms
'Update scheduled', 30.189000 ms
'Update scheduled', 40.084000 ms
'Update scheduled', 36.359000 ms
'Update scheduled', 37.280000 ms
'Update scheduled', 34.602000 ms
'Update scheduled', 31.507000 ms
'Update scheduled', 32.194000 ms
'Update scheduled', 29.746000 ms
'Update scheduled', 48.058000 ms
'Update scheduled', 38.987000 ms
'Update scheduled', 32.114000 ms
'Update scheduled', 43.881000 ms
'Update scheduled', 58.164000 ms
'Update scheduled', 33.340000 ms
'Update scheduled', 41.139000 ms
'Update scheduled', 37.981000 ms
'Update scheduled', 41.966000 ms
'Update sc